# Introduction to Programmatic Business Analytics Assignment
Utilizing the Bike Sharing Dataset from the UCI Machine Learning Repository, demonstrate your data science and machine learning skills to extract insights and predict bike rental demand. The dataset can be accessed directly via the provided URL and code.

In the cell below, we'll import all the packages needed for this assignment.

*Make sure you run this cell before proceeding with the rest of the notebook to avoid any import-related errors.*

In [2]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from zipfile import ZipFile
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

## Task 1: Data Wrangling and Exploration
### 1.1 Load the Dataset
Use the provided code to download and extract the dataset directly from the given URL into a pandas DataFrame.

In [3]:
# Download and extract the dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip'
zip_file = requests.get(url)
zip_file = ZipFile(BytesIO(zip_file.content))
df = pd.read_csv(zip_file.open('day.csv'))

### 1.2 Initial Inspection and Cleaning
Perform an initial inspection by methods like head, info, shape to understand the dataset's structure and content.
Check for missing values and handle them appropriately, if any.

In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
# to know which coulmns  we have and the name of them
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [6]:
# To get an idea how is our data
df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [7]:
# To know if there is any null in each columns
df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

### 1.3 data preparation
- 1.3.1 Convert dteday to a datetime object.
- 1.3.2 Extract year, month, day and daysofweek from dteday and add them as seperate columns to the dataframe.
- 1.3.3 Normalize the temp, atemp, hum, and windspeed features using MinMaxScaler
- 1.3.4 Encode the season, yr, mnth, holiday, weekday, workingday, and weathersit columns as categorical variables.
- 1.3.5 Check the data after the transformation

In [8]:
# 1.3.1 Converting dteday to a datetime object

df['dteday'] = pd.to_datetime(df['dteday'])
print(df.head())

   instant     dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1 2011-01-01       1   0     1        0        6           0   
1        2 2011-01-02       1   0     1        0        0           0   
2        3 2011-01-03       1   0     1        0        1           1   
3        4 2011-01-04       1   0     1        0        2           1   
4        5 2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

    cnt  
0   985  
1   801  
2  1349  
3  1562  
4  1600  


In [9]:

df['year'] = df['dteday'].dt.year
df['month'] = df['dteday'].dt.month
df['day'] = df['dteday'].dt.day
df['dayofweek'] = df['dteday'].dt.dayofweek  # Monday is 0, Sunday is 6

# Display the DataFrame with the new columns
print(df.head())

   instant     dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1 2011-01-01       1   0     1        0        6           0   
1        2 2011-01-02       1   0     1        0        0           0   
2        3 2011-01-03       1   0     1        0        1           1   
3        4 2011-01-04       1   0     1        0        2           1   
4        5 2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

    cnt  year  month  day  dayofweek  
0   985  2011      1    1          5  
1   801  2011 

In [10]:
# 1.3.2 Extract year, month, day and daysofweek from dteday and add them as seperate columns to the dataframe

df['year'] = df['dteday'].dt.year
df['month'] = df['dteday'].dt.month
df['day'] = df['dteday'].dt.day
df['dayofweek'] = df['dteday'].dt.dayofweek  # Monday is 0, Sunday is 6

# Display the DataFrame with the new columns
print(df.head())

   instant     dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1 2011-01-01       1   0     1        0        6           0   
1        2 2011-01-02       1   0     1        0        0           0   
2        3 2011-01-03       1   0     1        0        1           1   
3        4 2011-01-04       1   0     1        0        2           1   
4        5 2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

    cnt  year  month  day  dayofweek  
0   985  2011      1    1          5  
1   801  2011 

In [11]:
selected_columns_to_normal = ['temp', 'atemp', 'hum', 'windspeed']
scaler = MinMaxScaler()

# Fit and transform the selected columns
df[selected_columns_to_normal] = scaler.fit_transform(df[selected_columns_to_normal])

print(df.head())

   instant     dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1 2011-01-01       1   0     1        0        6           0   
1        2 2011-01-02       1   0     1        0        0           0   
2        3 2011-01-03       1   0     1        0        1           1   
3        4 2011-01-04       1   0     1        0        2           1   
4        5 2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.355170  0.373517  0.828620   0.284606     331         654   
1           2  0.379232  0.360541  0.715771   0.466215     131         670   
2           1  0.171000  0.144830  0.449638   0.465740     120        1229   
3           1  0.175530  0.174649  0.607131   0.284297     108        1454   
4           1  0.209120  0.197158  0.449313   0.339143      82        1518   

    cnt  year  month  day  dayofweek  
0   985  2011      1    1          5  
1   801  2011 

In [12]:
# 1.3.4 Encode the season, yr, mnth, holiday, weekday, workingday, and weathersit columns as categorical variables.
#since machine learning models work with numbers we have to make these columns as categorical

selected_columns_to_encode = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']

# Loop through each column and encode as category
for column in selected_columns_to_encode:
    df[column] = df[column].astype('category')

print(df.head())

   instant     dteday season yr mnth holiday weekday workingday weathersit  \
0        1 2011-01-01      1  0    1       0       6          0          2   
1        2 2011-01-02      1  0    1       0       0          0          2   
2        3 2011-01-03      1  0    1       0       1          1          1   
3        4 2011-01-04      1  0    1       0       2          1          1   
4        5 2011-01-05      1  0    1       0       3          1          1   

       temp     atemp       hum  windspeed  casual  registered   cnt  year  \
0  0.355170  0.373517  0.828620   0.284606     331         654   985  2011   
1  0.379232  0.360541  0.715771   0.466215     131         670   801  2011   
2  0.171000  0.144830  0.449638   0.465740     120        1229  1349  2011   
3  0.175530  0.174649  0.607131   0.284297     108        1454  1562  2011   
4  0.209120  0.197158  0.449313   0.339143      82        1518  1600  2011   

   month  day  dayofweek  
0      1    1          5  
1      1

### 1.4 visualizing the distribution of bike rentals (cnt) across different time granularities and categorical variables.
Conduct a comprehensive exploratory data analysis on the dataset, focusing on the distribution of bike rentals (cnt) across various dimensions and conditions. The analysis should reveal how bike rentals vary over different months, days of the week, seasons, holidays, working days, and weather situations.

The analysis should utilize Seaborn for creating visualizations, and these should be organized into a multi-panel figure using Matplotlib's subplot functionality. Each subplot is to represent a different aspect of the data:
1. Total bike rentals per month - Presented as a bar plot.
2. Total bike rentals per day of the week - Presented as a bar plot.
3. Bike rentals distribution per season - Displayed using a box plot.
4. Bike rentals on holidays vs. non-holidays - Illustrated with a box plot.
5. Bike rentals on working days vs. non-working days - Visualized through a box plot.
6. Bike rentals across different weather situations - Depicted with a box plot.

All plots should be appropriately titled and laid out for readability and visual appeal. Additionally, a consistent white grid style should be applied to all plots for uniformity

## Task 2: Statistical Analysis and Predictive Modeling

### 2.1 Descriptive and Inferential Statistical Analysis
Perform a detailed statistical analysis on a subset of features from the Bike Sharing dataset. The focus should be on understanding the relationships and characteristics of key numerical variables, including temperature (temp), feeling temperature (atemp), humidity (hum), wind speed (windspeed), and total bike rentals (cnt).

The following steps should be taken in the analysis:

1. Feature Selection: Extract a subset of the dataset containing only the columns temp, atemp, hum, windspeed, and cnt.
2. Descriptive Statistics: Generate the descriptive statistics (such as mean, median, standard deviation, etc.) for these selected features. This will provide an overview of the central tendency and spread of the data.
3. Inferential Statistics: Compute the correlation matrix for the selected features. This step is essential to understand how these variables are related to each other.
4. Visualization of Correlations: Use a heatmap to visualize the correlation matrix. The heatmap should be annotated and utilize a color scheme that clearly indicates the strength and direction of correlations.

All analyses and visualizations should be done with attention to detail, ensuring that the outputs are not only accurate but also easy to interpret. The heatmap, in particular, should be sized appropriately for clarity and aesthetic appeal.

### 2.2 Development and Evaluation of a Linear Regression Model
Execute a comprehensive regression analysis on the Bike Sharing dataset to predict bike rental counts (cnt). This task involves selecting relevant features, preparing the data, training a linear regression model, making predictions, and evaluating the model's performance.

The detailed steps to be followed in this task are:

1. Feature Selection: Choose appropriate predictor variables for the regression model. These should include season, yr (year), mnth (month), holiday, weekday, workingday, weathersit, temp (temperature), atemp (feeling temperature), hum (humidity), and windspeed.
2. Data Preparation: Prepare the dataset for training and testing. Assign the selected features to X and the target variable, bike rental counts (cnt), to y. Then, split the dataset into training and testing sets using an 80-20 split ratio and a set random state for reproducibility.
3. Model Training: Utilize the Linear Regression algorithm from the scikit-learn library to train the model on the training data.
4. Making Predictions: Use the trained model to make predictions on the test dataset.
5. Model Evaluation: Evaluate the model's performance by calculating key metrics such as the R^2 Score and Root Mean Squared Error (RMSE). The R^2 Score indicates the proportion of the variance in the dependent variable that is predictable from the independent variables, while RMSE provides a measure of the differences between values predicted by the model and the values actually observed.
The outcome of this task should be a well-trained Linear Regression model, along with a thorough evaluation of its predictive performance. All steps should be clearly documented, and findings should be reported with insights drawn from the performance metrics.

### 2.3 Clustering Analysis
Conduct a clustering analysis on the Bike Sharing dataset to uncover patterns and groupings within the environmental variables. The focus will be on the temperature (temp), feeling temperature (atemp), humidity (hum), and wind speed (windspeed) variables. This task involves several key steps, utilizing the KMeans clustering algorithm and evaluating the optimal number of clusters.

The analysis should proceed as follows:

1. *Feature Selection and Preparation:* Isolate the temp, atemp, hum, and windspeed features for clustering.
2. *Data Standardization:* Apply standard scaling to the selected features to normalize their range and variance, ensuring that each feature contributes equally to the clustering process.
3. *Determining Optimal Clusters:* Implement the silhouette method to find the optimal number of clusters. This involves running the KMeans algorithm with a varying number of clusters (from 2 to 10) and calculating the silhouette score for each. The silhouette score measures how similar an object is to its own cluster compared to other clusters.
4. *Silhouette Score Visualization:* Plot the silhouette scores against the number of clusters to visually determine the optimal cluster count.
5. *KMeans Clustering:* Perform KMeans clustering on the standardized data using the optimal number of clusters identified from the silhouette analysis.
6. *Integration with Original Data:* Append the cluster labels derived from KMeans to the original dataset, enabling further analysis based on the identified clusters.
The expected outcome is a well-defined set of clusters, each representing a unique combination of environmental conditions. The analysis should be presented with clear visualizations, particularly for the silhouette score plot, and detailed explanations of each step and its significance in the overall clustering process.

### 2.4 Visualizing Clustered Bike Rental Data Using Principal Component Analysis
Perform a dimensionality reduction and visualization on the previously clustered Bike Sharing dataset to better understand the clustering results. This task involves using Principal Component Analysis (PCA) to reduce the data to two dimensions and then creating a scatter plot to visualize the different clusters.

The specific steps to complete this task are:

1. Dimensionality Reduction with PCA: Implement Principal Component Analysis (PCA) to reduce the high-dimensional clustered data to two principal components. This step simplifies the dataset while retaining essential features necessary for understanding the clustering patterns.
2. Visualization of Clusters: After reducing the dimensions, use a scatter plot to visualize the clusters in this new two-dimensional space.
Plot Customization: Assign different colors to each cluster for clarity, label the axes as 'Principal Component 1' and 'Principal Component 2', and include a legend indicating the cluster numbers.
3. Interpretation of Results: Analyze the scatter plot to identify any distinct groupings or patterns that emerge among the clusters. Look for overlaps, distinct separations, or any other notable characteristics in the distribution of the clusters.

This task is aimed at providing a visual representation of the clustering results in a simpler, two-dimensional space, making it easier to interpret and analyze the relationships between different clusters. The outcome should be a clear and informative scatter plot that offers insights into the structure and distribution of the clustered data.

## Reflection:
Provide a brief reflection on what you learned, the challenges you faced, and how you overcame them.